In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.model_selection import LeaveOneOut
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from pyHSICLasso import HSICLasso
import collections

# Data preparation

In [91]:
y = pd.read_csv("train_data\severity_score_train.txt",sep = "\t")
cd4 = pd.read_csv("train_data\cd4_gene_expr_train.txt",sep = "\t")
cd8 = pd.read_csv("train_data\cd8_gene_expr_train.txt",sep = "\t")
cd19 = pd.read_csv("train_data\cd19_gene_expr_train.txt",sep = "\t")
nasal = pd.read_csv("train_data\\nasal_gene_expr_train.txt",sep = "\t")
nasal_m = pd.read_csv("train_data\\nasal_microbiome_train.txt",sep = "\t")
cd4 = cd4.T
cd8 = cd8.T
cd19 = cd19.T
nasal = nasal.T
nasal_m = nasal_m.T
test_cd4 = pd.read_csv("test_data\cd4_gene_expr_test.txt",sep = "\t")
test_cd8 = pd.read_csv("test_data\cd8_gene_expr_test.txt",sep = "\t")
test_cd19 = pd.read_csv("test_data\cd19_gene_expr_test.txt",sep = "\t")
test_nasal = pd.read_csv("test_data\\nasal_gene_expr_test.txt",sep = "\t")
test_nasal_m = pd.read_csv("test_data\\nasal_microbiome_test.txt",sep = "\t")
test_cd4 = test_cd4.T
test_cd8 = test_cd8.T
test_cd19 = test_cd19.T
test_nasal = test_nasal.T
test_nasal_m = test_nasal_m.T
y['group'] = pd.cut(y.severity_score,bins = [0,3,6,10],labels = [1,4,7])
y.index = y.subject_id
cd4.index = cd4.index.to_numpy().astype(np.int)
cd8.index = cd8.index.to_numpy().astype(np.int)
cd19.index = cd19.index.to_numpy().astype(np.int)
nasal.index = nasal.index.to_numpy().astype(np.int)
nasal_m.index = nasal_m.index.to_numpy().astype(np.int)
test_cd4.index = test_cd4.index.to_numpy().astype(np.int)
test_cd8.index = test_cd8.index.to_numpy().astype(np.int)
test_cd19.index = test_cd19.index.to_numpy().astype(np.int)
test_nasal.index = test_nasal.index.to_numpy().astype(np.int)
test_nasal_m.index = test_nasal_m.index.to_numpy().astype(np.int)

In [92]:
cd4_scaler = StandardScaler()
cd4_scaler.fit(cd4)
cd4.iloc[:,:] = cd4_scaler.transform(cd4)
test_cd4.iloc[:,:] = cd4_scaler.transform(test_cd4)

cd8_scaler = StandardScaler()
cd8_scaler.fit(cd8)
cd8.iloc[:,:] = cd8_scaler.transform(cd8)
test_cd8.iloc[:,:] = cd8_scaler.transform(test_cd8)

cd19_scaler = StandardScaler()
cd19_scaler.fit(cd19)
cd19.iloc[:,:] = cd19_scaler.transform(cd19)
test_cd19.iloc[:,:] = cd19_scaler.transform(test_cd19)

nasal_scaler = StandardScaler()
nasal_scaler.fit(nasal)
nasal.iloc[:,:] = nasal_scaler.transform(nasal)
test_nasal.iloc[:,:] = nasal_scaler.transform(test_nasal)

nasal_m_scaler = StandardScaler()
nasal_m_scaler.fit(nasal_m)
nasal_m.iloc[:,:] = nasal_m_scaler.transform(nasal_m)
test_nasal_m.iloc[:,:] = nasal_m_scaler.transform(test_nasal_m)



In [112]:
cd4_y = y.join(cd4,how = "left").sort_index()
cd8_y = y.join(cd8,how = "left").sort_index()
cd19_y = y.join(cd19,how = "left").sort_index()
nasal_y = y.join(nasal,how = "left").sort_index()
nasal_m_y = y.join(nasal_m,how = "left").sort_index()

In [113]:
cd4_y

,severity_score,subject_id,group,ENSG00000000419.12,ENSG00000001461.16,ENSG00000001497.16,ENSG00000001629.9,ENSG00000001631.14,ENSG00000002016.16,ENSG00000002549.12,...,ENSG00000280120.1,ENSG00000280137.1,ENSG00000280755.1,ENSG00000280778.1,ENSG00000280981.1,ENSG00000281350.1,ENSG00000281394.1,ENSG00000281485.1,ENSG00000281649.1,ENSG00000282031.1
subject_id,,,,,,,,,,,,,,,,,,,,,
101,7.617048,101,7,-1.932791,-1.127149,-1.108812,0.731581,0.129314,-1.238638,-1.310126,...,-0.781130,3.355209,1.634966,4.112461,-0.233505,-1.702211,-0.533320,-0.330401,-1.382725,-1.954839
104,4.983152,104,4,1.407797,0.527712,-0.432509,-0.476718,0.164440,-0.099975,0.175016,...,0.475959,-0.876123,1.335021,-0.019453,0.183302,0.241753,0.070659,-0.366962,-0.124524,2.304226
105,1.298928,105,1,0.259421,0.100131,1.071292,0.911246,0.371323,-0.887697,-0.585855,...,0.624084,0.144768,0.653409,-0.518883,-0.866701,-0.199402,-0.498434,-0.296526,2.329640,0.429111
108,6.806016,108,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,5.766088,109,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,7.793881,234,7,-1.101358,-0.384314,-1.988377,1.475334,0.157320,-0.876292,-1.021941,...,0.225690,1.878132,0.163082,0.319126,1.739019,0.348337,-0.533320,-0.330401,0.202250,-0.485838
235,3.546627,235,4,0.633512,0.017438,-0.286755,0.480100,0.390652,-0.118003,-0.003029,...,0.380543,0.169007,-0.361040,1.518036,2.343508,0.399196,-0.498434,3.063129,0.807228,-0.379263
236,0.621695,236,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Feature selection by HSICLasso

In [146]:
def feature_select(train_data,test_data,num):
    hsic_lasso = HSICLasso()
    hsic_lasso.input(train_data.iloc[:,3:].to_numpy(), train_data.iloc[:,2].to_numpy())
    hsic_lasso.classification(num)
    train_data_select = train_data.iloc[:,[0,1,2]+[x + 3 for x in hsic_lasso.get_index()]]
    test_data_select = test_data.iloc[:, hsic_lasso.get_index()]
    return train_data_select, test_data_select

In [147]:
train_data1, test_data1 = feature_select(cd4_y.dropna(),test_cd4,8)
train_data2, test_data2 = feature_select(cd8_y.dropna(),test_cd8,17)
train_data3, test_data3 = feature_select(cd19_y.dropna(),test_cd19,21)
train_data4, test_data4 = feature_select(nasal_y.dropna(),test_nasal,19)
train_data5, test_data5 = feature_select(nasal_m_y.dropna(),test_nasal_m,21)


Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Delta kernel for the outcomes.
Block HSIC Lasso B = 20.
M set to 1.
Using Gaussian kernel for the features, Delta kernel for the outcomes.
Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Delta kernel for the outcomes.
Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Delta kernel for the outcomes.
Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Delta kernel for the outcomes.


In [148]:
train_data_pool = [train_data1,
             train_data2,
             train_data3,
             train_data4,
             train_data5]
test_data_pool = [test_data1,
             test_data2,
             test_data3,
             test_data4,
             test_data5]

In [150]:
train_data_pool[1]

,severity_score,subject_id,group,ENSG00000119403.13,ENSG00000243708.8,ENSG00000277502.1,ENSG00000114784.3,ENSG00000130640.13,ENSG00000115904.12,ENSG00000011405.13,ENSG00000164022.16,ENSG00000112592.12,ENSG00000183617.4,ENSG00000124299.13,ENSG00000184277.12,ENSG00000258111.1,ENSG00000212461.1,ENSG00000111358.13,ENSG00000108439.9,ENSG00000089737.15
subject_id,,,,,,,,,,,,,,,,,,,,
101,7.617048,101,7,2.406874,0.091119,1.234312,-1.556558,-0.340184,0.011860,-0.719541,-0.925772,-0.875141,1.993075,-1.007233,-0.698745,-0.819598,0.483257,-0.282060,-1.662717,-1.382954
105,1.298928,105,1,-0.048565,-0.055742,-0.452602,-0.833977,-0.683929,-0.821504,-0.349384,-1.085808,1.315261,-0.897710,0.062381,-0.047780,-0.165902,-0.512689,-0.439015,0.038073,0.603347
108,6.806016,108,7,-0.352704,-0.191412,1.361391,0.629230,-0.546152,-0.625872,0.459369,0.580524,-0.668062,0.168468,-0.326973,-0.828821,0.256023,-0.512689,0.490869,-0.417365,-0.671600
126,2.717011,126,1,-1.537329,-0.461894,-0.528423,-0.668397,1.972395,-0.109894,2.158498,1.484610,-1.185839,-0.487828,-0.584534,-1.255906,-0.666149,-0.480967,-1.116723,-1.660132,-1.314762
127,9.280991,127,7,0.755424,-0.252978,-0.711185,-0.052560,0.662459,-0.837494,0.252446,0.767224,0.779886,0.805958,0.713736,-0.422359,0.756486,-0.242046,2.037375,-0.511256,0.163926
128,3.906028,128,4,-0.250713,1.257426,-0.711185,1.500644,-0.316390,1.605007,-0.630359,2.283061,-2.056709,2.595959,-1.071366,0.906453,-0.819598,-0.443390,0.798138,-0.506330,-1.564254
137,0.287199,137,1,-0.298654,-0.680777,-0.112338,-0.092884,-0.957254,-0.842787,-0.448582,-0.347603,0.857501,-1.024378,-0.303396,0.514815,-0.116717,-0.231721,-0.464436,1.172689,0.989813
138,5.662850,138,4,-0.641976,0.967190,-0.437918,0.976575,0.340218,1.798588,-0.632752,0.253882,-0.465007,-0.029966,-0.195575,0.893213,-0.429668,2.406167,-0.539873,-1.252648,-1.071056
143,2.795860,143,1,-1.541604,-1.070412,-0.711185,-0.280831,-1.159331,-1.916708,-0.949719,-0.325438,0.808086,-0.700514,-1.073112,2.592056,-0.819598,-0.443390,1.593939,0.614455,2.692214


In [151]:
train_miss = pd.DataFrame({   "severity_score":cd4_y["severity_score"],
                              "cd4":cd4_y.iloc[:,5],"cd8":cd8_y.iloc[:,5],
                              "cd19":cd19_y.iloc[:,5],
                              "nasal":nasal_y.iloc[:,5],
                              "nasal_m":nasal_m_y.iloc[:,5]})
train_miss.iloc[:,1:6] = train_miss.iloc[:,1:6].isna()

In [152]:
test_miss = pd.DataFrame({   
                              "cd4":test_cd4.iloc[:,5],"cd8":test_cd8.iloc[:,5],
                              "cd19":test_cd19.iloc[:,5],
                              "nasal":test_nasal.iloc[:,5],
                              "nasal_m":test_nasal_m.iloc[:,5]})
test_miss.iloc[:,0:5] = test_miss.iloc[:,0:5].isna()

# LOOCV of ensemble model

In [153]:
def have_train_data(id):
    data_dic = []
    for i in range(1,6):
        if not train_miss[train_miss.index == id].iloc[:,i].all():
            data_dic.append(i-1)
#     if any([x in data_dic for x in [0,1,2]]):
#         try: data_dic.remove(3)
#         except :pass
#         try: data_dic.remove(4)
#         except :pass
#     if 4 in data_dic:
#         try: data_dic.remove(4)
#         except :pas
#     if any([x in data_dic for x in [0,2]]):
#         try: data_dic.remove(1)
#         except :pass
    return data_dic

def run_one_data(data,data_key,subject_id):
    classifiers = [
    SVC(),
    LinearSVC(random_state=0),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=4),
    RandomForestClassifier(max_depth=4,random_state = 0),
    AdaBoostClassifier()]

    X_train = data[data_key].drop(subject_id).iloc[:,3:].to_numpy()
    y_train = data[data_key].drop(subject_id).iloc[:,2].to_numpy()
    X_test = data[data_key].loc[subject_id][3:].to_numpy().reshape(1, -1)
    predictions = []
    for i in classifiers:
        clf = OneVsRestClassifier(i)
        clf.fit(X_train, y_train)
        p = float(clf.predict(X_test))
        predictions.append(p)
        clf = OneVsOneClassifier(i)
        clf.fit(X_train, y_train)
        p = float(clf.predict(X_test))
        predictions.append(p)
    return predictions        

In [154]:
have_train_data(104)

[0, 3, 4]

In [155]:
loo = LeaveOneOut()
predictions = []
mse = 0
n = 0
for train_index, test_index in loo.split(train_miss):
    test_id = int(train_miss.index.to_numpy()[test_index])
    true_value = float(train_miss["severity_score"].iloc[test_index])
    data_dic = have_train_data(test_id)
    predictions = []
    if data_dic == []:
        predictions.append(4)
    else:
        for data_key in data_dic:
            predictions += run_one_data(train_data_pool,data_key,test_id)
#     if data_key in [4,5]:
#         final_prediction = 4
#     else:
#         final_prediction , _ = collections.Counter(predictions).most_common(1)[0]
    final_prediction , _ = collections.Counter(predictions).most_common(1)[0]
    print(true_value)
    print(predictions)
    #final_prediction = np.mean(predictions)
    n += 1
    mse += (final_prediction+ 0.5 - true_value) ** 2
mse/n

7.617048
[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 4.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 4.0, 7.0, 7.0, 7.0, 7.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0]
4.983152
[4.0, 4.0, 7.0, 7.0, 7.0, 7.0, 4.0, 7.0, 4.0, 4.0, 4.0, 7.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 7.0, 4.0, 4.0, 4.0, 4.0, 4.0]
1.298928
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 1.0, 7.0, 1.0, 1.0, 1.0, 1.0, 7.0, 4.0, 4.0, 1.0, 1.0, 4.0, 4.0, 4.0, 1.0, 4.0, 1.0, 1.0, 1.0]
6.8060160000000005
[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 1.0, 7.0]
5.766088
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 7.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 

5.406015
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]
0.592572
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.0, 1.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0]
4.905033
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 7.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 4.0, 4.0, 1.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 7.0, 1.0, 1.0, 1.0, 1.0, 4.0, 4.0, 1.0, 4.0, 4.0, 4.0, 4.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0]
6.950658
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 7.0, 4.0, 4.0, 4.0, 4.0, 4.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0, 7.0, 7.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0, 4.0]
5.9458470000000005
[1.0, 1.0, 1.0, 1.0, 7.0, 7.0, 4.0, 4.0, 4.0, 7.0, 1.0, 1.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]
5.208018
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]
4.7794300000000005
[4.0, 4.0, 

2.958953636904125

# Set up the model by all training data

In [157]:
model_pool = {}
index = 0
for data in train_data_pool:
    X = data.iloc[:,3:].to_numpy()
    y = data.iloc[:,2].to_numpy()
    model_pool[index] = [] 
    for model in [
    SVC(),
    LinearSVC(random_state=0),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=4),
    RandomForestClassifier(max_depth=4,random_state = 0),
    AdaBoostClassifier()]:
        model_pool[index].append(OneVsRestClassifier(model).fit(X, y))
        model_pool[index].append(OneVsOneClassifier(model).fit(X, y))
    index += 1
        
    


# Prediction

In [201]:
def predict(subject_id):
    data_dic = []
    for i in range(0,5):
        if not test_miss[test_miss.index == subject_id].iloc[:,i].all():
            data_dic.append(i)
    predictions = []
    if data_dic == []:
        return 4
    for data_key in data_dic:
        X_test = test_data_pool[data_key].loc[subject_id].to_numpy().reshape(1, -1)
        models = model_pool[data_key]
        for model in models:
            predictions.append(float(model.predict(X_test)))

    final_prediction , _ = collections.Counter(predictions).most_common(1)[0]   
    return final_prediction + 0.5
    
        
    

In [202]:
res = []
for i in test_miss.index.to_numpy():
    res.append(predict(i))

In [203]:
output = pd.DataFrame({"severity_score":res,"subject_id":test_miss.index.to_numpy()})

In [204]:
output.to_csv("prediction_0812.csv",index = False)

cd4         True
cd8         True
cd19        True
nasal       True
nasal_m    False
Name: 239, dtype: bool